# Introduction to Natural Language Processing with fastText
In this notebook we will discuss what is Natural Language Processing (NLP) and how to easily implement several projects using the library [fastText](https://github.com/facebookresearch/fastText).

In [2]:
#Load all libraries
import os,sys  
import pandas as pd
import numpy as np
import fasttext

print(sys.version)

3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
